In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns

import folium

import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
df = pd.read_csv('/content/doctors_data (2).csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/doctors_data (2).csv'

In [ ]:
sns.countplot(x="Rating",data=df)
plt.title('Ratings Countplot')
plt.savefig('ratings.png')
plt.show()


In [ ]:
px.set_mapbox_access_token("pk.eyJ1IjoibmVybWluYWhtZWQiLCJhIjoiY2xwNWE2ZGJuMXFnbTJpcnBvdG5iMHIzOSJ9.WlPqV1LBhk3ou73xHUZAGQ")
#configure_plotly_browser_state()
px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="Rating",
                   size_max=20, zoom=3, width=1000, height=1700)

In [ ]:
# Elbow method to determine the number of K in Kmeans Clustering
coords =df[['Longitude','Latitude']]

distortions = []
K = range(1,20)
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    distortions.append(kmeansModel.inertia_)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(K, distortions, marker='o')
plt.xlabel('k')
plt.ylabel('Distortions')
plt.title('Elbow Method For Optimal k')
plt.savefig('elbow.png')
plt.show()


In [1]:
from sklearn.metrics import silhouette_score

sil = []
kmin=2
kmax =20

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(coords)
  labels = kmeans.labels_
  sil.append(silhouette_score(coords, labels, metric = 'euclidean'))
plt.plot(range(kmin,kmax+1),sil)
plt.show()



NameError: name 'KMeans' is not defined

In [ ]:
sil



In [ ]:
kmeans = KMeans(n_clusters=15, init='k-means++')
kmeans.fit(coords)
y = kmeans.labels_
print("k = 15", " silhouette_score ", silhouette_score(coords, y, metric='euclidean'))

In [ ]:
df['cluster'] = kmeans.predict(df[['Longitude','Latitude']])
df



In [ ]:
px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="cluster",
                  hover_data= ['Doctors_Names', 'Latitude', 'Longitude'], zoom=10, width=1200, height=800)




**Location-Based Recommendation**

In [ ]:
top_doctors = df.sort_values(by=["Rating"], ascending=False)
top_doctors


In [ ]:
def recommend_Doctor(top_doctors, latitude,longitude):
    # Predict the cluster for longitude and latitude provided
    #x=df[['Latitude','Longitude'	]]
    cluster = kmeans.predict(np.array([latitude,longitude]).reshape(1,-1))[0]
    print(cluster)

    # Get the best doctor in this cluster
    return  top_doctors[top_doctors['cluster']==cluster].iloc[0:5][['Doctors_Names', 'Latitude','Longitude'	,'Location','Price']]





In [ ]:
test_coordinates = {
    'user': [1, 2, 3],
    'latitude' : [27.187669, 31.210678, 31.259465],
    'longitude' : [31.170370, 29.913089, 32.272630],
}

test_df = pd.DataFrame(test_coordinates)
test_df



In [ ]:
user1 = test_df[test_df['user'] == 1]
user1


In [ ]:
fig = px.scatter_mapbox( recommend_Doctor(top_doctors , user1.longitude, user1.latitude), lat="Latitude", lon="Longitude",
                   zoom=10, width=1200, height=800, hover_data= ['Doctors_Names', 'Latitude', 'Longitude','Price'])
fig.add_scattermapbox(lat=user1["latitude"], lon= user1["longitude"]).update_traces(dict(mode='markers', marker = dict(size = 15)))



In [ ]:
user2 = test_df[test_df['user'] == 2].reset_index()
fig = px.scatter_mapbox(recommend_Doctor(top_doctors, user2.longitude, user2.latitude), lat="Latitude", lon="Longitude",
                   zoom=10, width=1200, height=800, hover_data= ['Doctors_Names', 'Latitude', 'Longitude','Price'])
fig.add_scattermapbox(lat=user2["latitude"], lon= user2["longitude"]).update_traces(dict(mode='markers', marker = dict(size = 15)))


In [ ]:
user3 = test_df[test_df['user'] == 3].reset_index()
fig = px.scatter_mapbox(recommend_Doctor(top_doctors, user3.longitude, user3.latitude), lat="Latitude", lon="Longitude",
                   zoom=10, width=1200, height=800, hover_data= ['Doctors_Names', 'Latitude', 'Longitude','Price'])
fig.add_scattermapbox(lat=user3["latitude"], lon= user3["longitude"]).update_traces(dict(mode='markers', marker = dict(size = 15)))



In [ ]:
from google.colab import drive
drive.mount('/content/drive')